In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# Load the data
data = pd.read_csv('C:\\Users\\Admin\\Downloads\\Bootcamp Assignement\\NSSO68.csv', low_memory=False)

# Subset data for state 'KA'
df = data[['MPCE_URP','Whether_owns_any_land', 'hhdsz', 'Religion','Social_Group','Regular_salary_earner']]

# Check for missing values
print(df['MPCE_URP'].isna().sum())
print(df['Whether_owns_any_land'].isna().sum())
print(df['hhdsz'].isna().sum())
print(df['Religion'].isna().sum())
print(df['Social_Group'].isna().sum())
print(df['Regular_salary_earner'].isna().sum())

0
10
0
3
14
12


In [5]:
# Columns to impute
columns_to_impute = ['Whether_owns_any_land','Religion','Social_Group','Regular_salary_earner']

In [6]:
# Drop rows with any remaining NaN values
df.dropna(inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_15756\1461368480.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [7]:
# Check for missing values
print(df['MPCE_URP'].isna().sum())
print(df['Whether_owns_any_land'].isna().sum())
print(df['hhdsz'].isna().sum())
print(df['Religion'].isna().sum())
print(df['Social_Group'].isna().sum())
print(df['Regular_salary_earner'].isna().sum())

0
0
0
0
0
0


In [8]:
# Convert the target variable to binary based on the specified condition
df['MPCE_URP'] = df['MPCE_URP'].apply(lambda x: 0 if x < 380 else 1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_15756\2092469563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MPCE_URP'] = df['MPCE_URP'].apply(lambda x: 0 if x < 380 else 1)


In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

# Define the independent variables (X) and the dependent variable (y)
X = df[['Whether_owns_any_land', 'hhdsz', 'Religion','Social_Group','Regular_salary_earner']]  # replace with your actual column names
y = df['MPCE_URP']  # replace with your actual column name

# Add a constant term for the intercept
X = sm.add_constant(X)

# Define the Tobit model class
class Tobit(GenericLikelihoodModel):
    def __init__(self, endog, exog, left=0, right=np.inf, **kwargs):
        super(Tobit, self).__init__(endog, exog, **kwargs)
        self.left, self.right = left, right
    
    def nloglikeobs(self, params):
        exog = self.exog
        endog = self.endog
        left, right = self.left, self.right
        
        beta = params[:-1]
        sigma = params[-1]
        
        XB = np.dot(exog, beta)
        cens = (endog == left) * (left != -np.inf) + (endog == right) * (right != np.inf)
        uncens = 1 - cens
        
        ll = np.zeros(len(endog))
        
        ll[cens] = np.log(
            (1 / (np.sqrt(2 * np.pi) * sigma)) *
            np.exp(-((endog[cens] - XB[cens]) ** 2) / (2 * sigma ** 2))
        )
        
        ll[uncens] = np.log(
            (1 / (np.sqrt(2 * np.pi) * sigma)) *
            np.exp(-((endog[uncens] - XB[uncens]) ** 2) / (2 * sigma ** 2))
        )
        
        return -ll

    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwargs):
        if start_params is None:
            start_params = np.append(np.zeros(self.exog.shape[1]), 1)
        return super(Tobit, self).fit(start_params=start_params,
                                     maxiter=maxiter, maxfun=maxfun, **kwargs)

# Fit the Tobit model
tobit_model = Tobit(y, X)
tobit_results = tobit_model.fit()

# Print the summary of the model
print(tobit_results.summary())

C:\Users\Admin\AppData\Local\Temp\ipykernel_15756\3426349089.py:33: RuntimeWarning: invalid value encountered in log
  ll[cens] = np.log(
C:\Users\Admin\AppData\Local\Temp\ipykernel_15756\3426349089.py:38: RuntimeWarning: invalid value encountered in log
  ll[uncens] = np.log(
C:\Users\Admin\AppData\Local\Temp\ipykernel_15756\3426349089.py:38: RuntimeWarning: divide by zero encountered in log
  ll[uncens] = np.log(


Optimization terminated successfully.
         Current function value: -0.003281
         Iterations: 223
         Function evaluations: 362
                                Tobit Results                                 
Dep. Variable:               MPCE_URP   Log-Likelihood:                 333.38
Model:                          Tobit   AIC:                            -652.8
Method:            Maximum Likelihood   BIC:                            -586.1
Date:                Mon, 01 Jul 2024                                         
Time:                        23:05:37                                         
No. Observations:              101624                                         
Df Residuals:                  101618                                         
Df Model:                           5                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------

C:\Users\Admin\anaconda3\Lib\site-packages\statsmodels\base\model.py:2742: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
C:\Users\Admin\anaconda3\Lib\site-packages\statsmodels\base\model.py:2746: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")
